In [37]:
import chipwhisperer as cw
import matplotlib.pyplot as plt

try :
    scope.dis()
    target.dis()
except Exception as e:
    pass

n = 4
scope = cw.scope()
target = cw.target(scope)
scope.default_setup()
cw.program_target(scope, cw.programmers.STM32FProgrammer, "crt-CWNANO.hex")


See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5919 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5919 bytes


In [3]:
print(bin(0x5b))
print(bin(0x57))

0b1011011
0b1010111


In [38]:
import time
import numpy as np
scope.adc.samples = 100000
def reset_target(scope): 
    scope.io.nrst = 'low'
    time.sleep(0.01)
    scope.io.nrst = 'high'
    time.sleep(0.01)

def cap_pass_trace(pass_guess):
    ret = ""
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 16)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
#     target.write(pass_guess)
    target.simpleserial_write('s', pass_guess)

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace(as_int=True)
    return np.array(trace)

In [39]:
inp = bytearray([0xab,0x20,0x0,0x0])
trace = cap_pass_trace(inp)

In [40]:
r = target.simpleserial_read('r',16)

In [41]:
[int(i) for i in r]

[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [8]:
# %matplotlib tk
# # trace = np.array(cap_pass_trace(bytearray([])))
# plt.plot(trace)
# plt.show()
pattern = trace[25:35]#correct for begin mod_exp
pattern1 = trace[8780:8793]#correct mod_mul
pattern2 = trace[8748:8770]#correct end of for loop

In [30]:
from numpy import genfromtxt
# np.savetxt("mod_mul_pat.csv", pattern1, delimiter=",")
# pattern = genfromtxt('mod_mul_pat.csv', delimiter=',')

In [42]:
from tqdm.notebook import tqdm
def pattern_match(pat,trace):
    len_p = len(pat)

    l = []

    for i in tqdm(range(len(trace)-len_p)):
        temp = trace[i:i+len_p]
        cf = np.corrcoef(pat,temp)[0][1]
        l.append(cf)
    return l

In [43]:
pt = pattern_match(pattern,trace)
pt1 = pattern_match(pattern1,trace)
pt2 = pattern_match(pattern2,trace)

  0%|          | 0/99990 [00:00<?, ?it/s]

  0%|          | 0/99987 [00:00<?, ?it/s]

  0%|          | 0/99978 [00:00<?, ?it/s]

In [45]:
%matplotlib tk
from matplotlib.patches import Rectangle


fig, ax = plt.subplots(2,sharex=False)
ax[0].plot(trace)
count = 0
l = []
for i,val in enumerate(pt):
    if val>0.97:
        count+=1
        print(i,val)
        l.append(i)
        rect = Rectangle((i, 0), len(pattern),256,facecolor='orange')
        ax[0].add_patch(rect)
        
print(count)
count = 0
l1 = []
for i,val in enumerate(pt1):
    if val>0.97:
        count+=1
        print(i,val)
        l1.append(i)
        rect = Rectangle((i, 0), len(pattern1),256,facecolor='lime')
        ax[0].add_patch(rect)
        
print(count)
count = 0
l2 = []
for i,val in enumerate(pt2):
    if val>0.97:
        count+=1
        print(i,val)
        l2.append(i)
        rect = Rectangle((i, 0), len(pattern2),256,facecolor='salmon')
        ax[0].add_patch(rect)
        

        
ax[1].plot(trace)
print(count)
    
plt.show()

61 0.9981896858096401
7516 0.9982838713712459
15552 0.9931109910190568
18189 0.9891571948252046
20704 0.9957058323412288
23845 0.99591189043565
27368 0.9912811425939252
32107 0.9898945734239507
37051 0.9898221108670227
39645 0.9887219515995198
42002 0.9984189891332163
45522 0.9980094377119216
50149 0.9948930377860634
52954 0.9933615036576635
56696 0.9987664702412247
63930 0.9956345946744835
72816 0.990611523883231
76125 0.9914044149436613
79312 0.9959735695079638
83336 0.993932071703247
87406 0.9916063612072673
90043 0.9921503300730914
92558 0.9947278128653753
95910 0.9941420920631131
99812 0.9915706173207814
25
0
0


In [12]:
def find_val(l_mm,l_f):
    pre = 0
    num = ''
    for ele in l_f:
        temp = np.array(l_mm)
        val = len(temp[(temp>pre)*(temp<ele)])
        if val==2:
            num += '1'
        else :
            num+= '0'
        pre = ele
    num = num[::-1]
    print(num)
    print(int(num,2))

In [13]:
l.append(100000)
l1_ = np.array(l1)
l1_ = l1_[l1_<l[1]]
l2_ = np.array(l2)
l2_ = l2_[l2_<l[1]]
find_val(l1_,l2_)
l1_ = np.array(l1)
l1_ = l1_[l1_>l[1]]
l2_ = np.array(l2)
l2_ = l2_[l2_>l[1]]
find_val(l1_,l2_)

1011011
91
1010111
87


In [84]:
folder = "Our/"
np.savetxt(folder+"trace.csv", trace, delimiter=",")

In [14]:
import math

r = 2
e=3
n=17947
dp = 91
# p = 137
# q = 131
p = math.gcd(pow(r,e*(dp),n)-r%n+n,n)
print(p)

137


In [16]:
q = n//p
print(q)

131


In [32]:
import math
N = 1043347985599790750547972239032002691
e = 65537

c = 4
m = 396536242503841570983745957578538360


lis = [210, 89, 242, 74, 240, 168, 227, 8]

mp = 0
bits = ''
for i,val in enumerate(lis):
    mp += (1<<(8*i))*val
    
p = math.gcd(mp-m+N,N)
print(p)

1084024262488859977


In [36]:
q = N//p
print(q)
print(p*q)

962476599190059883
1043347985599790750547972239032002691
